In [ ]:
pip install mlxtend pandas

In [ ]:
import asyncio
import nest_asyncio
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
import pandas as pd

# Carregar produtos comprados

def recommend(user_product, rules):
    product_list = rules['antecedents'].explode().unique().tolist() + rules['consequents'].explode().unique().tolist()
    recommend = set()

    for product in user_product:
        items = rules[rules['antecedents'].apply(lambda x, product=product: product in x)]['consequents'].tolist()
        for item in items:
            recommend.update(item)
    recommend -= set(user_product)
    return list(recommend)

def main():
    product = pd.read_csv('https://raw.githubusercontent.com/gustavolace/MLpython/main/products.csv')
    sales = pd.read_csv('https://raw.githubusercontent.com/gustavolace/MLpython/main/sales.csv')
    product_sales = [products.split(',') for products in sales['produtos_comprados']]

    te = TransactionEncoder()
    te_ary = te.fit(product_sales).transform(product_sales)
    df_transacoes = pd.DataFrame(te_ary, columns=te.columns_)

    frequent_itemsets = apriori(df_transacoes, min_support=0.01, use_colnames=True)
    rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)

    print("Lista de Produtos:")
    print('\n'.join(product['produto'].tolist()))  # Imprimir lista com quebra de linha

    produtos_inseridos = input('\nInsira os produtos que você comprou (separados por vírgula): ')
    produtos_sugeridos = recommend(produtos_inseridos.split(','), rules)
    print(f'Produtos sugeridos: {produtos_sugeridos}')

if __name__ == '__main__':
    nest_asyncio.apply()
    main()
